## Setup

Importing dependencies:

In [2]:
import cobra
import pandas as pd
import random
import numpy as np
import os

## Load model

In [5]:
file_name = 'multi_omics_data.sbml'
file_name = 'data/iECIAI39_1322.xml'
model = cobra.io.read_sbml_model(file_name)

In [6]:
reaction = model.reactions[1000]
print(reaction.genes)
# gene = list(reaction.genes)[0]
# gene.annotation['uniprot']

frozenset({<Gene ECIAI39_0442 at 0x7fef0c359290>, <Gene ECIAI39_3506 at 0x7fef0c358a50>})


## Find list of reactions

In [7]:
def get_list_of_reactions(file_name):
    """

    :param file_name: Name of the model file (has to be xml for now)
    :return: None (prints the list of reactions that has mass in them)
    """

    # Load model¶depending on the kind of file (the file has to be xml)
    if file_name.endswith(".xml"):
        model = cobra.io.read_sbml_model(file_name)

    # Print out the reaction name and reaction id for all reactions related to BIOMASS production:
    print("List of reactions related to BIOMASS production:")
    for rxn in model.reactions:
        if rxn.name is not None and 'BIOMASS' in rxn.id:
            print("{}: {}".format(rxn.id, rxn.name))
            
# spits out the list of reaction names related to BIOMASS production
get_list_of_reactions(file_name)

List of reactions related to BIOMASS production:
BIOMASS_Ec_iJO1366_WT_53p95M: E. coli biomass objective function (iJO1366) - WT - with 53.95 GAM estimate
BIOMASS_Ec_iJO1366_core_53p95M: E. coli biomass objective function (iJO1366) - core - with 53.95 GAM estimate


Let's check if the reaction is bound:

In [8]:
# model.reactions.get_by_id('BIOMASS_Ecoli').bounds
model.reactions.get_by_id('BIOMASS_Ec_iJO1366_core_53p95M').bounds

(0.0, 1000.0)

It isn't, and that is a good thing

Let's fix glucose to a set input:

In [9]:
model.reactions.get_by_id('EX_glc__D_e').lower_bound = -15
model.reactions.get_by_id('EX_glc__D_e').upper_bound = -15

In [10]:
model.reactions.get_by_id("EX_glc__D_e")

Reaction identifier,EX_glc__D_e
Name,D-Glucose exchange
Memory address,0x07fef0c25a610
Stoichiometry,glc__D_e <-- D-Glucose <--
GPR,
Lower bound,-15
Upper bound,-15


Let's calculate fluxes now:

In [11]:
model.summary()

In [12]:
solution = model.optimize()

In [13]:
model.summary()

We can always check what a metabolite is by doing this:

In [14]:
for met in model.metabolites:
    if "gly" in met.id:
        print("{}: {}".format(met.id, met.name))
model.metabolites.get_by_id('glyclt_e')

glycogen_c: Glycogen C6H10O5
gly_c: Glycine
cgly_e: Cys Gly C5H10N2O3S
gly_e: Glycine
glyald_e: D-Glyceraldehyde
glyb_e: Glycine betaine
glyc_e: Glycerol
glyc__R_e: (R)-Glycerate
glyc2p_e: Glycerol 2-phosphate
glyc3p_e: Glycerol 3-phosphate
glyclt_e: Glycolate C2H3O3
manglyc_e: 2(alpha-D-Mannosyl)-D-glycerate
progly_e: L-Prolinylglycine
glyc_c: Glycerol
glyald_c: D-Glyceraldehyde
urdglyc_c: (-)-Ureidoglycolate
cgly_c: Cys Gly C5H10N2O3S
progly_c: L-Prolinylglycine
glyc3p_c: Glycerol 3-phosphate
glyb_c: Glycine betaine
cgly_p: Cys Gly C5H10N2O3S
glyc_p: Glycerol
glyc2p_c: Glycerol 2-phosphate
glyc2p_p: Glycerol 2-phosphate
bglycogen_c: Branching glycogen
glyald_p: D-Glyceraldehyde
glyb_p: Glycine betaine
glyc3p_p: Glycerol 3-phosphate
glyc__R_c: (R)-Glycerate
glyc__R_p: (R)-Glycerate
glyclt_c: Glycolate C2H3O3
glyclt_p: Glycolate C2H3O3
glytrna_c: Glycyl-tRNA(Gly)
trnagly_c: TRNA(Gly)
gly_p: Glycine
manglyc_p: 2(alpha-D-Mannosyl)-D-glycerate
2pglyc_c: 2-Phosphoglycolate
progly_p: L-Prol

Metabolite identifier,glyclt_e
Name,Glycolate C2H3O3
Memory address,0x07fef11a1fd90
Formula,C2H3O3
Compartment,e
In 2 reaction(s),"GLYCLTtex, EX_glyclt_e"


Define a file path for input/output files

In [15]:
data_file_path = './data'
if not os.path.isdir(data_file_path):
    os.mkdir(data_file_path)

Get the optimized solution to the model given constraint of amount of glucose production
Takes in the model and a reaction id which we are trying to optimize

In [16]:
def get_optimized_solution(model, reaction_id):
    """

    :param model:
    :param reaction_id:
    :return solution:
    """

    # check to see if the reaction is bound (need to add a check here)
    # DISCUSS!!
    print(model.reactions.get_by_id(reaction_id).bounds)

    # fix the flux value to -15 as we have data for this constraint
    model.reactions.get_by_id(reaction_id).lower_bound = -15
    model.reactions.get_by_id(reaction_id).upper_bound = -15
    model.reactions.get_by_id(reaction_id)

    solution = model.optimize()

    return solution

Generate fake data for proteomics, transcriptomics and metabolomics

In [17]:
def generate_mock_data(model, condition):
    """

    :param model: cobra model object
    :param solution: solution for the model optimization using cobra
    :param data_type: defines the type of -omics data to generate (all by default)
    :return:
    """

    # reaction_id of choice passed to the function# hardcoded here for this particular file (Need to convert this to an interactive cli program)
    reaction_id = 'BIOMASS_Ec_iJO1366_core_53p95M'
    # solution = None

    while condition:
        print("Condition parameter: ", condition)
        condition-=1
        solution = get_optimized_solution(model, reaction_id)

        get_proteomics_transcriptomics_data(model, solution, condition)

        get_metabolomics_data(model, condition)

In [18]:
def write_data_files(dataframe, data_type=None, condition=1):
    """

    :param dataframe:
    :param data_type:
    :param condition:
    :return:
    """
    # print(dataframe)
    # print("\n")

    # create the filenames
    sample_file_name = f'{data_file_path}/sample_{condition}.csv'
    omics_file_name = f'{data_file_path}/{data_type}_fakedata_sample_{condition}.csv'

    # Write the dataframe into a csv file
    # dataframe.to_csv(file_name, sep=',', encoding='utf-8')

    # create file number one: sample file
    if not os.path.isfile(sample_file_name):
        try:
            with open(sample_file_name, 'w') as fh:
                fh.write("Line Name,\n")
                fh.write(f"Sample {condition}")
        except Exception as ex:
            print("Error in writing file!")
            print(ex)

    # create file number two: omics file
    # TODO: Need to change the units to actual relevant units
    unit_dict = { "proteomics": 'g/L',\
            "transcriptomics": "g/L",\
            "metabolomics": "g/L"
            }

    try:
        with open(omics_file_name, 'w') as fh:
            fh.write("Line Name, Measurement Type, Value, Units\n")
            sample_name = f"Sample {condition}\n"
            spaces = ' '*len(sample_name)
            fh.write(sample_name)
            for index, series in dataframe.iteritems():
                for id, value in series.iteritems():
                    fh.write((f"{spaces}{id}, {value}, {unit_dict[data_type]}\n"))

    except Exception as ex:
        print("Error in writing file!")
        print(ex)


### Get fake proteomics and transcriptomics data by assuming linear relationship
### Pi = qi.Ti => therefore Ti = Pi/qi + ∂
### Vi = Ki.Pi => therefore Pi = Vi/Ki + ß
### ignoring the noise here

In [19]:
def get_proteomics_transcriptomics_data(model, solution, condition):
    """

    :param model:
    :param solution:
    :param condition:
    :return:
    """

    # pre-determined linear constant (NOTE: Allow user to set this via parameter)
    # DISCUSS!!
    k = 0.8
    q = 0.6

    proteomics = {}
    transcriptomics = {}

    rxnIDs = solution.fluxes.keys()
    for rxnId in rxnIDs:
        reaction = model.reactions.get_by_id(rxnId)
        for gene in list(reaction.genes):

            # this will ignore all the reactions that does not have the gene.annotation property
            # DISCUSS!!
            if gene.annotation:
                if 'uniprot' not in gene.annotation:
                    protein_id = gene.annotation['goa']
                else:
                    protein_id = gene.annotation['uniprot']

                # create proteomics dict
                proteomics[protein_id] = solution.fluxes[rxnId]/k

            # create transcriptomics dict
            transcriptomics[gene.id] = proteomics[protein_id]/q


    file_name = f'{data_file_path}/proteomics_fakedata_condition_{condition}.csv'
    proteomics_dataframe = pd.DataFrame.from_dict(proteomics, orient='index')
    write_data_files(proteomics_dataframe, "proteomics")

    file_name = f'{data_file_path}/transcriptomics_fakedata_condition_{condition}.csv'
    transcriptomics_dataframe = pd.DataFrame.from_dict(transcriptomics, orient='index')
    write_data_files(transcriptomics_dataframe, "transcriptomics")


In [20]:
def get_metabolomics_data(model, condition):
    """

    :param model:
    :param condition:
    :return:
    """
    metabolomics = {}
    # get metabolites
    # NOTE: Need to find a better algorithm. This is O(n^3)
    for met in model.metabolites:
        # get associated reactions
        for reaction in list(met.reactions):
            # get dictionary of associated metabolites and their concentrations
            for metabolite, conc in reaction._metabolites.items():
                if metabolite.id == met.id:
                    if met.id not in metabolomics.keys():
                        metabolomics[met.id] = abs(conc)
                    else:
                        metabolomics[met.id] += abs(conc)
        # getting number of associated reactions and averaging the metabolic concentration value
        num_reactions = len(list(met.reactions))
        metabolomics[met.id]/=num_reactions

    metabolomics_dataframe = pd.DataFrame.from_dict(metabolomics, orient='index')
    # Write the dataframe into a csv file
    file_name = f'{data_file_path}/metabolomics_fakedata_condition_{condition}.csv'
    write_data_files(metabolomics_dataframe, "metabolomics")

## Call the fake data generator

In [21]:
condition = 1
generate_mock_data(model, condition)

Condition parameter:  1
(0.0, 1000.0)


cobra/util/solver.py:416 UserWarning: solver status is 'infeasible'
